# This notebook demos a shallow water stencil

We start with some imports & a small helper function to convert our stencil to SIR:

In [1]:
from typing import Callable

from inspect import getsource
import ast

from dusk.grammar import Grammar

from dawn4py import compile, CodeGenBackend
from dawn4py.serialization import make_sir, SIR
from dawn4py.serialization.SIR import GridType
from dawn4py._dawn4py import run_optimizer_sir


def dusk_to_sir(stencil: Callable) -> SIR:
    # this will give wrong line numbers, there should be a way to fix them
    name = stencil.__name__
    stencil = ast.parse(getsource(stencil))

    assert isinstance(stencil, ast.Module)
    assert len(stencil.body) == 1
    stencil = stencil.body[0]
    assert Grammar.is_stencil(stencil)

    return make_sir(
        name, GridType.Value("Unstructured"), [Grammar().stencil(stencil)]
    )

With the above definitions, we can write our shallow water stencil:

In [2]:
from dusk.script import *


@stencil
def shallow_water(hC: Field[Cell], hC_t: Field[Cell],
                     vC: Field[Cell], vC_t: Field[Cell],
                     uC: Field[Cell], uC_t: Field[Cell],
                     hC_x: Field[Cell], hC_y: Field[Cell], uvC_div: Field[Cell],
                     hE: Field[Edge], vE: Field[Edge], uE: Field[Edge],
                     nx: Field[Edge], ny: Field[Edge], L: Field[Edge], alpha: Field[Edge],
                     boundary_edges: Field[Edge], boundary_cells: Field[Cell],
                     A: Field[Cell], edge_orientation: Field[Cell > Edge],
                     Grav: Field[Cell]):

    with levels_downward:

        # lerp cell quantities to edges
        hE = sum_over(Edge > Cell, hC, weights=[1-alpha, alpha])
        uE = sum_over(Edge > Cell, uC, weights=[1-alpha, alpha])
        vE = sum_over(Edge > Cell, vC, weights=[1-alpha, alpha])
    
        # boundary conditions on cells
        if (boundary_edges):
            uE = 0.
            vE = 0.

        # height field gradient
        hC_x = sum_over(Cell > Edge, hE * nx * L * edge_orientation)
        hC_y = sum_over(Cell > Edge, hE * ny * L * edge_orientation)

        # height field gradient is zero on the boundaries
        if (boundary_cells):
            hC_x = 0.
            hC_y = 0.

        # divergence of velocity field
        uvC_div = sum_over(Cell > Edge, (uE*nx + vE*ny)
                           * edge_orientation * L) / A

        # build ODE's
        uC_t = -Grav * hC_x
        vC_t = -Grav * hC_y
        hC_t = hC * uvC_div   


Then we can use dusk to convert the stencil to SIR. With dawn we can compile SIR to C++ which we will write to `shallow_water_cxx-naive.cpp`:

In [4]:
sir = dusk_to_sir(shallow_water)
cpp_naive = compile(sir, backend=CodeGenBackend.CXXNaiveIco)
with open("shallow_water_cxx-naive.cpp", "w+") as f:
    f.write(cpp_naive)

The generated C++ code also requires a driver which is already setup for this demo. With the driver code we can generate an executable `runner`:

In [5]:
!make

First, we put the runner into test mode to ensure that the computed kernel is correct:

In [6]:
!./runner test

mesh stats: #cells 680 #nodes 378 #edges 1057
congratulations!, your stencil is correct, you can visualize now!


If the tester reported that your dusk stencil works correctly, you can now run the complete stencil and visualize. It takes quite a while to run. Its finished after 4000 time steps.

In [7]:
!./runner run

mesh stats: #cells 680 #nodes 378 #edges 1057
time 0.04 timestep 20 dt 0.002
time 0.08 timestep 40 dt 0.002
time 0.12 timestep 60 dt 0.002
time 0.16 timestep 80 dt 0.002
time 0.2 timestep 100 dt 0.002
time 0.24 timestep 120 dt 0.002
time 0.28 timestep 140 dt 0.002
time 0.32 timestep 160 dt 0.002
time 0.36 timestep 180 dt 0.002
time 0.4 timestep 200 dt 0.002
time 0.44 timestep 220 dt 0.002
time 0.48 timestep 240 dt 0.002
time 0.52 timestep 260 dt 0.002
time 0.56 timestep 280 dt 0.002
time 0.6 timestep 300 dt 0.002
time 0.64 timestep 320 dt 0.002
time 0.68 timestep 340 dt 0.002
time 0.72 timestep 360 dt 0.002
time 0.76 timestep 380 dt 0.002
time 0.8 timestep 400 dt 0.002
time 0.84 timestep 420 dt 0.002
time 0.88 timestep 440 dt 0.002
time 0.92 timestep 460 dt 0.002
time 0.96 timestep 480 dt 0.002
time 1 timestep 500 dt 0.002
time 1.04 timestep 520 dt 0.002
time 1.08 timestep 540 dt 0.002
time 1.12 timestep 560 dt 0.002
time 1.16 timestep 580 dt 0.002
time 1.2 timestep 600 dt 0.002
time 1

In [8]:
%%capture
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.animation as animation
import glob
from IPython.display import HTML

def update_plot(frame_number, zarray, plot):
    z = np.loadtxt(out_files[frame_number])    
    plot[0].remove()    
    plot[0] = ax1.plot_trisurf(x, y, z, triangles=T-1, cmap=plt.cm.winter)    
    
zarray=[]

out_files = sorted(glob.glob('out/stepH*.txt'))

x,y = np.loadtxt('initP.txt', unpack=True)
T = np.loadtxt('initT.txt')
z = np.loadtxt('initC.txt')
fig = plt.figure(figsize=(12,8), dpi=100)
ax1 = fig.add_subplot(111, projection='3d')  
ax1.set_zlim(1.5,2.5)
ax1.set_title('Shallow Water')
plot = [ax1.plot_trisurf(x, y, np.zeros(np.size(z)), triangles=T-1, cmap=plt.cm.autumn)]


In [9]:
animate = animation.FuncAnimation(fig, update_plot, len(out_files), fargs=(zarray, plot), interval = 50)
HTML(animate.to_html5_video())